In [51]:
#############
#############  Team Member :
#############  Name : Md Ziauddin Ridoy
#############  Matriculation Number : 220100676
#############    

In [1]:
import findspark
findspark.init()

import pyspark

In [3]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc, col, sum

In [7]:
from pyspark.sql import SparkSession


spark= SparkSession.builder.appName('dataframe').getOrCreate()

In [9]:
# load the retail dataset:
retail_data=spark.read.csv("online-retail-dataset.csv",inferSchema=True,header=True)

In [10]:
retail_data.show()

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|12/1/2010 8:26|     7.65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|12/1/2010 8:26|     4.

In [11]:
# # Which item was bought most (total) ?
best_selling_items = retail_data.groupBy(col("StockCode")).agg(sum(col("Quantity")).alias("sum_quantity")).sort(desc("sum_quantity"))
best_selling_items.show(5)


+---------+------------+
|StockCode|sum_quantity|
+---------+------------+
|    22197|       56450|
|    84077|       53847|
|   85099B|       47363|
|   85123A|       38830|
|    84879|       36221|
+---------+------------+
only showing top 5 rows



In [14]:
# # Description of the item which was bought most
retail_data.select("StockCode", "Description").where(col("StockCode") == "22197").dropDuplicates().show()

+---------+--------------------+
|StockCode|         Description|
+---------+--------------------+
|    22197|SMALL POPCORN HOLDER|
|    22197|      POPCORN HOLDER|
+---------+--------------------+



In [17]:
# # which item was bought most in the USA?
best_selling_items_usa = retail_data.where(col("Country") == "USA").groupBy(col("StockCode")).agg(sum(col("Quantity")).alias("sum_quantity")).sort(desc("sum_quantity"))
best_selling_items_usa.show(5)

+---------+------------+
|StockCode|sum_quantity|
+---------+------------+
|    23366|          88|
|    20975|          72|
|    21121|          48|
|    21124|          48|
|    21123|          48|
+---------+------------+
only showing top 5 rows



In [18]:
# # Description of the item which was bought most in USA
retail_data.select("StockCode", "Description").where(col("StockCode") == "23366").dropDuplicates().show()

+---------+--------------------+
|StockCode|         Description|
+---------+--------------------+
|    23366|SET 12 COLOUR PEN...|
|    23366|SET 12 COLOURING ...|
|    23366|SET 12 COLOURING ...|
+---------+--------------------+



In [19]:
# # Which was the lowest invoice (>0), which one the highest?
all_invoices = retail_data.groupBy(col("InvoiceNo")).agg(sum(col("UnitPrice")*col("Quantity")).alias("total_cost")).sort(col("total_cost"))
all_invoices.show()

+---------+----------+
|InvoiceNo|total_cost|
+---------+----------+
|  C581484| -168469.6|
|  C541433|  -77183.6|
|  C556445|  -38970.0|
|  C550456|  -22998.4|
|  C580605| -17836.46|
|  C540117| -16888.02|
|  C540118| -16453.71|
|  C537651| -13541.33|
|  C537630| -13541.33|
|  C537644| -13474.79|
|  C570556| -11816.64|
|  C580604|  -11586.5|
|  A563186| -11062.06|
|  A563187| -11062.06|
|  C573079|  -8322.12|
|  C574902|  -8286.22|
|  C551685|  -8142.75|
|  C566899|  -7427.97|
|  C553355|  -7006.83|
|  C551699|   -6930.0|
+---------+----------+
only showing top 20 rows



In [20]:
# # postive invoices ; lowest invoice / Highest invoice
postive_invoices = all_invoices.where(col("total_cost") > 0)
postive_invoices.show()



+---------+----------+
|InvoiceNo|total_cost|
+---------+----------+
|   570554|      0.38|
|   567869|       0.4|
|   539441|      0.42|
|   542736|      0.55|
|   573589|      0.55|
|   540833|      0.65|
|   560217|      0.79|
|   542136|      0.84|
|   546876|      0.84|
|   549534|      0.84|
|   544438|      0.85|
|   551538|      0.85|
|   544927|      0.85|
|   540945|      0.85|
|   539645|      0.95|
|   538194|      0.95|
|   538669|      0.95|
|   572832|       1.0|
|   558055|      1.07|
|   564665|       1.1|
+---------+----------+
only showing top 20 rows



In [21]:
# # Add a column which displays whether an item was purchased in Germany
retail_data_germany = retail_data.withColumn("BroughtInGermany", col("Country") == "Germany")
retail_data_germany.show()


+---------+---------+--------------------+--------+--------------+---------+----------+--------------+----------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|BroughtInGermany|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+----------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|           false|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|           false|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|           false|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|           false|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|           false|
|   536365|    22752|SET 7 BABUSHKA NE..

In [30]:
# Purchased items in Germany only ##(Using Where)
purchases_items_germany = retail_data_germany.where(col("Country") == "Germany").where(col("BroughtInGermany") == "true")
purchases_items_germany.show(10)

+---------+---------+--------------------+--------+---------------+---------+----------+-------+----------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|Country|BroughtInGermany|
+---------+---------+--------------------+--------+---------------+---------+----------+-------+----------------+
|   536527|    22809|SET OF 6 T-LIGHTS...|       6|12/1/2010 13:04|     2.95|     12662|Germany|            true|
|   536527|    84347|ROTATING SILVER A...|       6|12/1/2010 13:04|     2.55|     12662|Germany|            true|
|   536527|    84945|MULTI COLOUR SILV...|      12|12/1/2010 13:04|     0.85|     12662|Germany|            true|
|   536527|    22242|5 HOOK HANGER MAG...|      12|12/1/2010 13:04|     1.65|     12662|Germany|            true|
|   536527|    22244|3 HOOK HANGER MAG...|      12|12/1/2010 13:04|     1.95|     12662|Germany|            true|
|   536527|    22243|5 HOOK HANGER RED...|      12|12/1/2010 13:04|     1.65|     12662|

In [35]:
# Purchased items in Germany only ##(Using filter)
purchased_items_different_way = retail_data_germany.filter( (retail_data_germany.Country == "Germany") & (retail_data_germany.BroughtInGermany == "true") )
purchased_items_different_way.show(10)

+---------+---------+--------------------+--------+---------------+---------+----------+-------+----------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|Country|BroughtInGermany|
+---------+---------+--------------------+--------+---------------+---------+----------+-------+----------------+
|   536527|    22809|SET OF 6 T-LIGHTS...|       6|12/1/2010 13:04|     2.95|     12662|Germany|            true|
|   536527|    84347|ROTATING SILVER A...|       6|12/1/2010 13:04|     2.55|     12662|Germany|            true|
|   536527|    84945|MULTI COLOUR SILV...|      12|12/1/2010 13:04|     0.85|     12662|Germany|            true|
|   536527|    22242|5 HOOK HANGER MAG...|      12|12/1/2010 13:04|     1.65|     12662|Germany|            true|
|   536527|    22244|3 HOOK HANGER MAG...|      12|12/1/2010 13:04|     1.95|     12662|Germany|            true|
|   536527|    22243|5 HOOK HANGER RED...|      12|12/1/2010 13:04|     1.65|     12662|

In [40]:
# #Add a column which shows the total amount of corresponding invoice:
retail_data.join(all_invoices, ["InvoiceNo"]).show()


+---------+---------+--------------------+--------+---------------+---------+----------+--------------+------------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|        total_cost|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+------------------+
|   536596|    21624|VINTAGE UNION JAC...|       1|12/1/2010 17:29|     5.95|      null|United Kingdom|             38.09|
|   536596|    22900| SET 2 TEA TOWELS...|       1|12/1/2010 17:29|     2.95|      null|United Kingdom|             38.09|
|   536596|    22114|HOT WATER BOTTLE ...|       1|12/1/2010 17:29|     3.95|      null|United Kingdom|             38.09|
|   536596|    21967|PACK OF 12 SKULL ...|       1|12/1/2010 17:29|     0.29|      null|United Kingdom|             38.09|
|   536596|   84926A|WAKE UP COCKEREL ...|       4|12/1/2010 17:29|     1.25|      null|United Kingdom|             38.09|
|   536596|    2

In [47]:
# #How many German customers spent more than $10?
invoices_germany = retail_data.where(col("Country") == "Germany").groupBy(col("InvoiceNo")).agg(sum(col("UnitPrice")*col("Quantity")).alias("total_cost")).sort(col("total_cost"))
#invoices_germany.show()
more_than_10 = invoices_germany.where(col("total_cost") > 10).count()
print(more_than_10)

455


In [49]:
# # Sort the German customers with respect to their total invoice in descending order: 
german_customers = retail_data.where(col("Country") == "Germany").groupBy(col("InvoiceNo"), col("CustomerID")).agg(sum(col("UnitPrice")*col("Quantity")).alias("total_cost")).sort(desc("total_cost"))
german_customers.show()

+---------+----------+------------------+
|InvoiceNo|CustomerID|        total_cost|
+---------+----------+------------------+
|   552978|     12590| 9341.260000000004|
|   564856|     12477|4257.0599999999995|
|   543737|     12477|2355.9500000000003|
|   581179|     12471|2238.5099999999993|
|   550137|     12477|           2221.02|
|   540458|     12501|2169.3899999999994|
|   569640|     12471| 2145.380000000001|
|   576910|     12477|2037.6899999999996|
|   538174|     12471|1982.4099999999999|
|   557466|     13815|1965.7400000000002|
|   571065|     12613|           1881.86|
|   578472|     12476|           1750.46|
|   541965|     12625|           1716.62|
|   571328|     12473|           1697.98|
|   537201|     12472|1631.3000000000009|
|   539395|     12471|1484.2499999999998|
|   547822|     12668|            1444.8|
|   550527|     12476|1411.6699999999998|
|   569844|     12626|1377.5499999999993|
|   571824|     12472|1376.4299999999998|
+---------+----------+------------

In [50]:
spark.stop()